In [1]:
import strats

import betfairlightweight as bflw
from flumine import Flumine, clients, BaseStrategy
from betfairlightweight.filters import streaming_market_filter, streaming_market_data_filter
from flumine import Flumine, clients

import os
from dotenv import load_dotenv
import time
import logging
from pythonjsonlogger import jsonlogger
import pandas as pd
from betfairlightweight.filters import market_filter

# load .env
load_dotenv()

USERNAME = os.getenv("BETFAIR_USERNAME")
APP_KEY  = os.getenv("BETFAIR_APP_KEY")
PASSWORD = os.getenv("BETFAIR_PASSWORD")


trading = bflw.APIClient(USERNAME, app_key=APP_KEY, password=PASSWORD, cert_files=("certs/client-2048.crt", "certs/client-2048.key"))  # you still need a live session/login
trading.login()


market = market_filter(
    event_type_ids=["4"],     # 4 = Cricket
    market_type_codes=["MATCH_ODDS"],
    market_countries=["GB", "IN"],
)

catalogues = trading.betting.list_market_catalogue(
    filter=market,
    max_results=100,
    market_projection=["MARKET_START_TIME","RUNNER_DESCRIPTION","EVENT"]
)

for m in catalogues:
    print(m.json())
    for r in m.runners:
        print("  ", r.selection_id, r.runner_name)

# Only cricket (eventTypeId = 4)
flt = market_filter(event_type_ids=["4"])
countries = trading.betting.list_countries(filter=flt)

for c in countries:
    print(c.country_code, c.market_count)


b'{"marketId":"1.247647159","marketName":"Match Odds","marketStartTime":"2025-09-12T17:30:00.000Z","totalMatched":27783944.29,"runners":[{"selectionId":10301,"runnerName":"England","handicap":0.0,"sortPriority":1},{"selectionId":349,"runnerName":"South Africa","handicap":0.0,"sortPriority":2}],"event":{"id":"34719737","name":"England v South Africa","countryCode":"GB","timezone":"GMT","openDate":"2025-09-12T17:30:00.000Z"}}'
   10301 England
   349 South Africa
b'{"marketId":"1.247680577","marketName":"Match Odds","marketStartTime":"2025-09-12T23:00:00.000Z","totalMatched":11598.8,"runners":[{"selectionId":40497467,"runnerName":"Barbados Royals","handicap":0.0,"sortPriority":1},{"selectionId":11056084,"runnerName":"Trinbago Knight Riders","handicap":0.0,"sortPriority":2}],"event":{"id":"34723019","name":"Barbados Royals v Trinbago Knight Riders","countryCode":"GB","timezone":"GMT","openDate":"2025-09-12T23:00:00.000Z"}}'
   40497467 Barbados Royals
   11056084 Trinbago Knight Riders
b'

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)  # Set to logging.CRITICAL to speed up simulation

client  = clients.BetfairClient(trading, paper_trade=True)
framework = Flumine(client)

strategy= strats.HugoStrat(   
    market_filter=market,
    market_data_filter=streaming_market_data_filter(ladder_levels=3),
    max_order_exposure=1000,
    max_selection_exposure=150,
    context={"stake": 4},
    log_root="./logs/live_paper/",
    log_level="D")

framework.add_strategy(strategy)
framework.run()


KeyboardInterrupt: 